**Group 22**

Name  | Surname | Email  
---------|-------------------|---------
Julio|Vigueras|20220661@novaims.unl.pt 
Ariel|Pérez|20220662@novaims.unl.pt
Miguelanguel|Mayuare|20220665@novaims.unl.pt
Ayotunde|Aribo|20221012@novaims.unl.pt

# Model handcrafted "D"
---

## Increase of model complexity

Increasing the complexity of a CNN model can be advantageous in improving its ability to capture complex patterns and dependencies in image data, which can lead to better performance in classification. However, increasing the number of layers and filters in a CNN model should be done with caution to prevent overfitting to the training data. We already applied regularization with droputs layers and data augmentation. Larger models with more parameters may require more computational resources and longer training times, which can be a limitation in practical applications.

In this model with added and extra Conv2D layer per block and a fully connected Dense layer of 256 units.

The model performed very well, nevertheless, the added cost must be weighted considering that the dataset is small and the model C performed slightly worse with lower complexity.

In [ ]:
# Make the imports
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory

import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from math import floor

import pathlib

2023-04-08 20:27:08.932566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 20:27:09.495490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# get the data
!wget https://www.dropbox.com/s/n3320qxwdn3rs19/moths.zip?dl=0 -O moths.zip
!unzip moths.zip

First, the previous model is imported in order to make the necessary modifications.

A new model can be developed from scratch using this model as template.

In [ ]:
def model_D(blocks=4, input_shape=(224, 224, 3)):
    data_augmentation = keras.Sequential([
        layers.RandomRotation(0.2), 
        layers.RandomFlip(),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
        layers.RandomZoom(0.1),
    ])

    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    for i in range(blocks + 1, blocks + 5):
        x = layers.Conv2D(filters=2**i, kernel_size=3, padding="same",
                          kernel_initializer=initializers.GlorotNormal(seed=123), 
                          activation="relu")(x)
        residual = x
        x = layers.Conv2D(filters=2**i, kernel_size=3, padding="same",
                          kernel_initializer=initializers.GlorotNormal(seed=123), 
                          activation="relu")(x)
        x = layers.MaxPooling2D(pool_size=2, padding="same")(x)
        residual = layers.Conv2D(2**i, 1, strides=2)(residual)
        x = layers.add([x, residual])
        x = layers.Dropout(0.3)(x)
    residual = x
    x = layers.Conv2D(filters=256, kernel_size=3, 
                      activation="relu", padding="same")(x)
    residual = layers.Conv2D(2**i, 1, strides=2)(residual)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(30, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = model_D(4)
model.summary()

2023-04-08 19:38:23.249875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 19:38:23.273977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 19:38:23.274151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['sequential[0][0]']             
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 32  896         ['rescaling[0][0]']          

In [ ]:
# Compile model 
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer='adam', 
              metrics=["accuracy"])

In [ ]:
dataset_path = pathlib.Path("moths")
input_shape = (224, 224, 3)
batch_size = 64

In [ ]:
# Split datasets
train_dataset = image_dataset_from_directory(
    dataset_path / "train",
    image_size=input_shape[:2],
    batch_size=batch_size)
validation_dataset = image_dataset_from_directory(
    dataset_path / "valid",
    image_size=input_shape[:2],
    batch_size=batch_size)
test_dataset = image_dataset_from_directory(
    dataset_path / "test",
    image_size=input_shape[:2],
    batch_size=batch_size)

Found 3558 files belonging to 30 classes.
Found 445 files belonging to 30 classes.
Found 408 files belonging to 30 classes.


In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="saved_models/model_handcrafted_D.keras",
        save_best_only=True,
        monitor="val_loss")
]

In [ ]:
# Train model
history = model.fit(
    train_dataset,
    epochs=100,
    batch_size=batch_size,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/100


2023-04-08 19:38:40.029155: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3558]
	 [[{{node Placeholder/_4}}]]
2023-04-08 19:38:40.029344: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [3558]
	 [[{{node Placeholder/_4}}]]
2023-04-08 19:38:41.184506: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-08 19:38:42.069005: I tensorflow/compiler/xla

56/56 [==============================] - ETA: 0s - loss: 3.5497 - accuracy: 0.0385

2023-04-08 19:39:19.621764: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [445]
	 [[{{node Placeholder/_4}}]]
2023-04-08 19:39:19.622053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [445]
	 [[{{node Placeholder/_0}}]]


56/56 [==============================] - 41s 431ms/step - loss: 3.5497 - accuracy: 0.0385 - val_loss: 3.3838 - val_accuracy: 0.0607
Epoch 2/100
56/56 [==============================] - 15s 259ms/step - loss: 3.2203 - accuracy: 0.0745 - val_loss: 2.8728 - val_accuracy: 0.1371
Epoch 3/100
56/56 [==============================] - 15s 262ms/step - loss: 2.9164 - accuracy: 0.1164 - val_loss: 2.7433 - val_accuracy: 0.1573
Epoch 4/100
56/56 [==============================] - 15s 262ms/step - loss: 2.7285 - accuracy: 0.1644 - val_loss: 2.6452 - val_accuracy: 0.1933
Epoch 5/100
56/56 [==============================] - 15s 261ms/step - loss: 2.5146 - accuracy: 0.2187 - val_loss: 2.2200 - val_accuracy: 0.3236
Epoch 6/100
56/56 [==============================] - 15s 261ms/step - loss: 2.4379 - accuracy: 0.2406 - val_loss: 2.0742 - val_accuracy: 0.3461
Epoch 7/100
56/56 [==============================] - 15s 260ms/step - loss: 2.2514 - accuracy: 0.3075 - val_loss: 1.9547 - val_accuracy: 0.3753
Epoc

In [ ]:
# Visualization
hist_df = pd.DataFrame(history.history)
loss = px.scatter(hist_df['loss'])
val_loss = px.line(hist_df['val_loss'])
accuracy = px.scatter(hist_df['accuracy'])
val_accuracy = px.line(hist_df['val_accuracy'])

fig = make_subplots(cols=2, rows=1, subplot_titles=("Loss", "Accuracy"))
fig.add_trace(loss.data[0], col=1, row=1)
fig.add_trace(val_loss.data[0], col=1, row=1)
fig.add_trace(accuracy.data[0], col=2, row=1)
fig.add_trace(val_accuracy.data[0], col=2, row=1)

fig.show()

![Accuracy and loss](https://www.dropbox.com/s/hcegod9l1v3b61u/modeld.png?raw=1)

Comparing the plots from the previous model, this is much better because it took longer to overfit.

Below are both models compared with the test set.

In [ ]:
model_A = load_model("saved_models/model_handcrafted_A.keras")
model_B = load_model("saved_models/model_handcrafted_B.keras")
model_C = load_model("saved_models/model_handcrafted_C.keras")
model_D = load_model("saved_models/model_handcrafted_D.keras")

_, model_A_acc = model_A.evaluate(test_dataset)
_, model_B_acc = model_B.evaluate(test_dataset)
_, model_C_acc = model_C.evaluate(test_dataset)
_, model_D_acc = model_D.evaluate(test_dataset)

print(
    f"Model A: {model_A_acc * 100:.2f}% of accuracy\n"
    f"Model B: {model_B_acc * 100:.2f}% of accuracy\n"
    f"Model C: {model_C_acc * 100:.2f}% of accuracy\n"
    f"Model D: {model_D_acc * 100:.2f}% of accuracy")

2023-04-08 20:28:02.037318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [408]
	 [[{{node Placeholder/_0}}]]
2023-04-08 20:28:02.037550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [408]
	 [[{{node Placeholder/_0}}]]
2023-04-08 20:28:02.847636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


4/7 [================>.............] - ETA: 0s - loss: 1.1104 - accuracy: 0.7266

2023-04-08 20:28:03.544596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


7/7 [==============================] - 2s 156ms/step - loss: 0.4946 - accuracy: 0.8627
Model A: 69.85% of accuracy
Model B: 64.95% of accuracy
Model C: 82.60% of accuracy
Model D: 86.27% of accuracy


### Model comparisson

Even with the added complexity, the models behave similarly, the intuiton says that is because the dataset is very small, containing few samples per class.
